#### 크롤링 (4가지 방법)

1. requests
    - API 이용 : string을 json으로
    - Html 이용 : string을 html으로, bs4 이용

2. selenium : 직접 브라우져 띄워서 가져옴

3. scrapy : 웹 크롤링 위해 만들어진 파이썬 패키지

#### 크롤링 권장 방법 순서

- API이용 : html의 필요없는 엘리먼트 이름 많아서 API 이용하는게 속도 더 빠르다)
- html : 그냥 데이터 가지고 와서 웹 브라우져 띄우는 selenium 보다 빠르다) 
- Selenium


- - - 

### 1. requests
#### 01. API 이용

##### what is API?

API는 프로그램과 또 다른 프로그램을 연결해주는 일종의 다리

In [1]:
import requests

In [ ]:
# response = request.get(url)
# json_info = resoponse.json()
# json_info[][] 사용하기
# return json_normalize() : 데이터 프레임으로 만들기
# return 정보 단순 출력

#### 네이버 주식 데이터 가져오기

In [4]:
import pandas as pd
from pandas.io.json import json_normalize

In [7]:
# 페이지 사이즈와 페이지 값을 받자

def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize="+str(pageSize)+"&page="+str(page)

In [21]:
# 데이터를 가지고 와서 판다스의 데이터 프레임으로 만드는 함수

def get_data(url):
    response = requests.get(url) # get으로 데이터를 요청해서 string타입으로 받는다.
    json_info = response.json() # string을 json 포맷으로 parsing한다.
    companies = json_info['result']['itemList']
    return json_normalize(companies)

# print(json_info)
# print(companies) # 리스트 안에 딕셔너리 데이터 타입으로 한 회사의 정보가 들어가 있다.
    

# 아래 방법처럼 for문 사용할 수도 있지만, normalize로 한번에 넣어 줄 수 있다.

# df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"]) # 컬럼 명을 먼저 만들고
#     for company in companies:
#         df.loc[len(df)] = {
#             "종목":company["nm"],
#             "시세":company["nv"],
#             "전일비":company["cv"],
#             "등락율":company["cr"],
#             "시가총액":company["mks"],
#             "거래량":company["aq"],}
    
    
    

In [28]:
url = make_url(1500, 1)
df = get_data(url)
df.tail()

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
1453,0,82,244820,0.38,40,21,0,ARIRANG 스마트베타4종결합,10490,10450,2
1454,4,503,222190,1.23,115,19,0,ARIRANG 스마트베타 Momentum,9490,9375,2
1455,0,51,295820,0.68,65,15,0,ARIRANG 200동일가중,9685,9620,2
1456,0,51,295860,1.31,125,14,0,ARIRANG 200모멘텀,9660,9535,2
1457,0,50,295890,0.52,50,14,0,ARIRANG 200로우볼,9585,9535,2


- - -
#### dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api이다.
- https://api.darksky.net/


In [29]:
FORECAST_TOKEN = "d42b9b5217897e99db3140753294b90c"

In [49]:
# 1. 위도와 경도를 입력받아서 직접 가지고 오는 함수 만들기
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    # print(json_info)
    return json_info["timezone"] +" weather : " + json_info['hourly']['summary']


In [50]:
lat = 37.593901
lng = 127.014369
forecast(lat, lng)

'Asia/Seoul weather : Partly cloudy starting later this evening, continuing until tomorrow morning.'

In [52]:
# 2. 이미 만들어 놓은 패키지 이용하기

- - - 
#### 02. html 과 bs4 이용

In [51]:
from bs4 import BeautifulSoup

#### 네이버 키워드 랭킹 데이터 크롤링

In [98]:
#dom.select() : 여러개의 html 엘리먼트 셀렉팅 시 사용, 결과로 리스트 데이터를 리턴
#dom.select_one() : 하나의 html 엘리먼트 셀렉팅 시 사용, 결과로 문자열 데이터를 리턴

def naver():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content,"html.parser")
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")
    
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    
    #len(keywords)로 20개만 뽑히는지 확인하기!
    return df

naver_df = naver()
naver_df["keyword"]

0             조재현
1             모로코
2        포르투갈 모로코
3             복지로
4        재일교포 여배우
5       2001년 시트콤
6            포르투갈
7           라디오스타
8             이명희
9     포르투갈 모로코 중계
10       모로코 피파랭킹
11         아동수당신청
12         모로코 위치
13            벨한다
14           훈남정음
15         모로코 감독
16         월드컵 b조
17           암라바트
18            윤서인
19      강진 여고생 실종
Name: keyword, dtype: object

In [99]:
import time
ls = []
for idx in range(3):
    print(idx)
    ls.append(naver()) #ls에 데이터 프레임이 하나하나 들어간다.
    time.sleep(5) # 초단위

0
1
2


In [119]:
# 실시간 검색어 크롤링하는 함수 만들기
# 뭐든지 함수로 구현하는 것이 좋다!

def naver():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content,"html.parser")
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item > .ah_a")
    
    for keyword in keywords:
    
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
            
        }
    
    return df
    

In [121]:
naver_df = naver()

In [123]:
naver_df["keyword"]

0          포르투갈 모로코
1               조재현
2               모로코
3               복지로
4          재일교포 여배우
5         2001년 시트콤
6             라디오스타
7             조영남교수
8               이명희
9         김비서가 왜그럴까
10      포르투갈 모로코 중계
11           아동수당신청
12             포르투갈
13        강진 여고생 실종
14    김비서가 왜그럴까 몇부작
15    김비서가 왜그럴까 ost
16         모로코 피파랭킹
17           모로코 위치
18           월드컵 b조
19           꽃보다 할배
Name: keyword, dtype: object

- - - 

#### 다시 한번 더 연습하기

requests를 이용해서 html을 가지고 온다.
html을 string 형태로 가지고 온 후 BeautifulSoup으로 html parsing 해준다.
가지고 온 html을 이용해서 data프레임에 원하는 정보를 집어 넣는다.

In [131]:
# naver의 실시간 검색어 크롤링해서 data frame에 넣어주는 함수 구현하기

def naver():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content,"html.parser")
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")
    
    for keyword in keywords:
        
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    
    return df

naver_df = naver()    

In [133]:
naver_df
naver_df["keyword"]

0      포르투갈 모로코
1           모로코
2           조재현
3           복지로
4          베나티아
5         라디오스타
6     2001년 시트콤
7      재일교포 여배우
8            페페
9        스페인 이란
10     우루과이 사우디
11         한끼줍쇼
12         포르투갈
13          모나코
14       아동수당신청
15       월드컵 b조
16          이명희
17     모로코 피파랭킹
18        조영남교수
19        호날두 골
Name: keyword, dtype: object

- - -
#### 다음의 실시간 이슈 검색어 10개 수집하기

In [173]:
# 다음 실시간 이슈 검색어 크롤링 후 데이터 프레임에 넣는 함수 만들기

def daum():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".realtime_part > .list_hotissue.issue_row > li")
    
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text[:-1],
            "keyword":keyword.select_one(".link_issue").text
        }
    
    return df

In [177]:
daum_df = daum()
daum_df

,rank,keyword
0,1,포르투갈 모로코
1,2,조재현
2,3,이명희
3,4,유아
4,5,추적 60분
5,6,페페
6,7,재일교포 여배우
7,8,라디오 스타
8,9,로맨스 패키지
9,10,식량일기


In [176]:
daum_df["keyword"]

0    포르투갈 모로코
1         조재현
2         이명희
3          유아
4      추적 60분
5          페페
6    재일교포 여배우
7      라디오 스타
8     로맨스 패키지
9        식량일기
Name: keyword, dtype: object

#### 네이버, 다음 실시간 검색어에 공통된 키워드 찾기

In [179]:
result = [
    keyword for keyword in naver_df["keyword"] if daum_df["keyword"].str.contains(keyword).any()
]

result
# 8개의 키워드가 중복 됨

['포르투갈 모로코', '모로코', '조재현', '재일교포 여배우', '페페', '포르투갈', '이명희']

#### file download : requests

- 파일의 url을 받아서 해당 url의 파일을 다운로드 하는 방법

#### 네이버 영화 평점 크롤링하기

In [ ]:
def movie():
    df = pd.DataFrame(columns=["rate","date","comment"])
    
    response = requests.get("https://m.search.naver.com/search.naver?query=%EC%A5%AC%EB%9D%BC%EA%B8%B0+%EC%9B%94%EB%93%9C%3A+%ED%8F%B4%EB%A5%B8+%ED%82%B9%EB%8D%A4&sm=mtb_sug.top&where=m&oquery=%EC%A5%AC%EB%9D%BC%EA%B8%B0&tqi=TyVDMlpySoGssclc32lssssstV0-421517&qdt=0&acq=%EC%A5%AC%EB%9D%BC%EA%B8%B0+&acr=1#api=%3Fwhere%3Dbridge%26sm%3Digr_brg%26query%3D%25EC%25A5%25AC%25EB%259D%25BC%25EA%25B8%25B0%2520%25EC%259B%2594%25EB%2593%259C%253A%2520%25ED%258F%25B4%25EB%25A5%25B8%2520%25ED%2582%25B9%25EB%258D%25A4%26tab_prs%3Dcsa%26col_prs%3Dcsa%26nqx_theme%3D%257B%2522theme%2522%253A%257B%2522main%2522%253A%257B%2522name%2522%253A%2522movie_info%2522%252C%2522pkid%2522%253A%252268%2522%257D%257D%257D%26format%3Dtext%26tab%3Dpoint%26x_csa%3D%257B%2520%2522movieId%2522%253A%2522154285%2522%2520%252C%2522type%2522%253A%2522single%2522%2520%257D&_lp_type=cm")
    dom = BeaufitulSoup(response,"html.parser")
    